# Configuracion de grafica a usar

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# Importacion de librerias

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from IPython.display import display, clear_output
from ipywidgets import interact, IntSlider
import h5py
import numpy as np
import random

%matplotlib inline
import sys
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from LTC import *
!wget -q https://raw.githubusercontent.com/JefeLitman/VideoDataGenerator/master/DatasetsLoaderUtils.py -O DatasetsLoaderUtils.py
from DatasetsLoaderUtils import load_videoFrames_from_path

# Configuraciones para Tensorflow y Keras

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
tf.debugging.set_log_device_placement(False)

In [ ]:
random.seed(8128)
np.random.seed(8128)
tf.random.set_seed(8128)

# Carga de Datos

In [ ]:
root_path = "/home/jefelitman/DataSets/ucf101/split_1"
root_path

In [ ]:
epoch = 20
batch_size = 30
size = [112, 112]
frames = 16
canales = 3
video_shape = tuple([frames]+size[::-1]+[canales])

### Version antiguo con VideoDataGenerator

### Version nueva con load_videoFrames

In [ ]:
dataset = load_videoFrames_from_path(root_path, lambda x: x, size)

In [ ]:
train_gen = dataset.data_generator(1, canales)
test_gen = dataset.data_generator(2, canales)

In [ ]:
def train_gen_sampling():
    for v, l in train_gen:
        paso = len(v)//16
        if paso == 0:
            video = np.concatenate([v, np.zeros([16 - len(v), 112, 112, 3])], axis=0)
        else:
            video = v
        for j in range(paso):
            yield video[j::paso][:16], l
def test_gen_sampling():
    for v, l in test_gen:
        paso = len(v)//16
        if paso == 0:
            video = np.concatenate([v, np.zeros([16 - len(v), 112, 112, 3])], axis=0)
        else:
            video = v
        for j in range(paso):
            yield video[j::paso][:16], l

In [ ]:
107039/30 # Cantidad de batches

In [ ]:
def scale(video, label):
    return video/255., label

In [ ]:
def video_flip_horizontal(video, label):
    return tf.reverse(video, [2]), label

In [ ]:
os.mkdir("cache")
# Train dataset
train_data = tf.data.Dataset.from_generator(train_gen_sampling, (tf.float32, tf.int64), 
                                            (video_shape, []))
train_data = train_data.cache('cache/train').map(scale, 24)
#train_data = train_data.concatenate(train_data.map(video_flip_horizontal, 24))

#  Test dataset
test_data = tf.data.Dataset.from_generator(test_gen_sampling, (tf.float32, tf.int64), 
                                            (video_shape, []))
test_data = test_data.cache("cache/test").map(scale, 24)

# Red Neuronal LTC

### Construccion del modelo

In [ ]:
#Entrada de la red neuronal
dropout = 0.5
lr = 1e-3
weigh_decay = 5e-3

ltc_save_path = '/home/jefelitman/Saved_Models/trained_ucf/No_Encoder/no_inception/LTC-ori_split1_{w}x{h}x{f}_SGD_'.format(
        w=size[0], h=size[1],f=frames)
if canales == 3:
    ltc_save_path += 'RGB_'
else:
    ltc_save_path += 'B&N_'

ltc_save_path += 'lr={l}_DO_IC_TDM_S255_VFH_E{e}'.format(l = lr, e = epoch)

#Creacion de la carpeta donde se salvara el modelo
if not os.path.isdir(ltc_save_path):
    os.mkdir(ltc_save_path)
model_saves_path = os.path.join(ltc_save_path,'model_saves')
if not os.path.isdir(model_saves_path):
    os.mkdir(model_saves_path)
ltc_save_path

In [ ]:
#Parametros para la compilacion del modelo
optimizador = keras.optimizers.SGD(learning_rate=lr, momentum=0.9)
#optimizador = keras.optimizers.Adam(learning_rate=lr)
perdida = keras.losses.SparseCategoricalCrossentropy(name="loss")
precision = keras.metrics.SparseCategoricalAccuracy(name="acc")

In [ ]:
ltc = get_LTC_original(video_shape, len(dataset.to_class), dropout, weigh_decay)

In [ ]:
#Compilacion del modelo
ltc.compile(optimizer = optimizador,
           loss = perdida,
           metrics = [precision])

In [ ]:
#keras.utils.plot_model(ltc, 'LTC.png', show_shapes=True)

In [ ]:
#ltc = keras.models.load_model(os.path.join(ltc_save_path,'ltc_final_1000.h5'))

In [ ]:
ltc.summary()

### Cargo los pesos pre entrenados

##### Pesos del C3D

##### Pesos de la InceptionV3

### Entrenamiento de la red con el generador

In [ ]:
#Funcion customizadas para el entrenamiento del modelo 
def cambio_lr(epoch, lr):
    if epoch == 10 or epoch == 16 :
        for i in ['conv3d_1','conv3d_2','conv3d_3','conv3d_4', 'conv3d_5','dense_6','dense_7','dense_8']:
            weigh_decay = ltc.get_layer(i).kernel_regularizer.get_config()['l2'] * 0.1
            ltc.get_layer(i).kernel_regularizer = keras.regularizers.l2(weigh_decay)
        return optimizador.learning_rate.numpy() * 0.1
    else:
        return optimizador.learning_rate.numpy()

funciones = [
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(model_saves_path,'ltc_epoch_{epoch}.h5'),
        save_best_only=True,
        monitor='val_acc',
        verbose=1),
    keras.callbacks.LearningRateScheduler(cambio_lr, verbose=1),
    keras.callbacks.CSVLogger(os.path.join(ltc_save_path,'output.csv'))
]

In [ ]:
train_data = train_data.shuffle(len(dataset.__videos_train_path__),
                                reshuffle_each_iteration=True).batch(batch_size).prefetch(1)

test_data = test_data.shuffle(len(dataset.__videos_test_path__),
                              reshuffle_each_iteration=True).batch(batch_size).prefetch(1)

historial = ltc.fit(x = train_data,
                 epochs=epoch,
                 callbacks=funciones,
                 validation_data=test_data)

In [ ]:
!rm -rf cache

### Guardado del modelo 

In [ ]:
#Salvado final definitivo del modelo una vez se detenga
ltc.save(os.path.join(ltc_save_path,"ltc_final_{e}.h5".format(e=epoch)))

### Graficas de los resultados de entrenamiento

In [ ]:
fig = plt.figure()
plt.plot(historial.history["loss"],'k--')
plt.plot(historial.history["val_loss"],'b--')
plt.title('Loss over epochs')
plt.legend(labels=["Loss","Val_Loss"])
plt.show()
fig.savefig(os.path.join(ltc_save_path,'train_loss_epochs_{e}.png'.format(e=epoch)))

In [ ]:
fig = plt.figure()
plt.plot(historial.history["acc"],'k--')
plt.plot(historial.history["val_acc"],'b--')
plt.title('Accuracy over epochs')
plt.legend(labels=["Accuracy","Val_Accuracy"])
plt.show()
fig.savefig(os.path.join(ltc_save_path,'train_accuracy_epochs_{e}.png'.format(e=epoch)))

### Evaluacion del entrenamiento